In [ ]:
import pandas as pd
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from collections import Counter

#import utk tokenizing
import nltk
nltk.download("punkt")

#install utk stemming
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 215kB 5.5MB/s 


In [ ]:
soal = pd.read_excel("/content/drive/MyDrive/PER-TA-AN DUNIAWI/Data/42-01/Question.xlsx")
soal.head()

,Question,Label
0,Aktor yang bertanggung jawab atas analisis pre...,C1
1,Fokusnya terhadap Enterprise/Executive IS adal...,C1
2,Sistem pemrosesan transaksi yang secara konsta...,C1
3,Komponen yang dikembangkan oleh Gartner dalam ...,C3
4,Fokus utama pengembangan sistem informasi adal...,C1


In [ ]:
#casefold
soal["Casefold"] = soal["Question"].str.lower()
print(soal["Casefold"].head())

0    aktor yang bertanggung jawab atas analisis pre...
1    fokusnya terhadap enterprise/executive is adal...
2    sistem pemrosesan transaksi yang secara konsta...
3    komponen yang dikembangkan oleh gartner dalam ...
4    fokus utama pengembangan sistem informasi adal...
Name: Casefold, dtype: object


In [ ]:
#tokenizing

#remove punctuation
def remove_punctuation(text):
  return text.translate(str.maketrans("","",string.punctuation))
soal["Tokenize"] = soal["Casefold"].apply(remove_punctuation)

#remove whitespace
def remove_whitespace_LT(text):
  return text.strip()
soal["Tokenize"] = soal["Tokenize"].apply(remove_whitespace_LT)

#NLTK word tokenize
def word_tokenize_wrapper(text):
  return word_tokenize(text)
soal["Tokenize"] = soal["Tokenize"].apply(word_tokenize_wrapper)

print(soal["Tokenize"].head())

0    [aktor, yang, bertanggung, jawab, atas, analis...
1    [fokusnya, terhadap, enterpriseexecutive, is, ...
2    [sistem, pemrosesan, transaksi, yang, secara, ...
3    [komponen, yang, dikembangkan, oleh, gartner, ...
4    [fokus, utama, pengembangan, sistem, informasi...
Name: Tokenize, dtype: object


In [ ]:
#NLTK calc frequency distribution
#utk menghitung frekuensi distribusi token pada tiap row data
def freqDist_wrapper(text):
  return FreqDist(text)
soal["FreqDist"] = soal["Tokenize"].apply(freqDist_wrapper)

print(soal["FreqDist"].head().apply(lambda x: x.most_common()))

0    [(yang, 2), (analisis, 2), (aktor, 1), (bertan...
1    [(fokusnya, 1), (terhadap, 1), (enterpriseexec...
2    [(sistem, 1), (pemrosesan, 1), (transaksi, 1),...
3    [(komponen, 1), (yang, 1), (dikembangkan, 1), ...
4    [(fokus, 2), (adalah, 2), (dan, 2), (pada, 2),...
Name: FreqDist, dtype: object


In [ ]:
#stopwords

#get stopword indonesia
list_stopwords = stopwords.words('indonesian')

#additional stopword manually
list_stopwords.extend(["adalah", "yaitu"])

#convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pd list token
def stopwords_removal(words):
  return [word for word in words if word not in list_stopwords]
soal["Stopwords"] = soal["Tokenize"].apply(stopwords_removal)

print(soal["Stopwords"].head())

0    [aktor, bertanggung, analisis, prediktif, anal...
1    [fokusnya, enterpriseexecutive, is, sejarah, b...
2    [sistem, pemrosesan, transaksi, konstan, terli...
3    [komponen, dikembangkan, gartner, menguraikan,...
4    [fokus, utama, pengembangan, sistem, informasi...
Name: Stopwords, dtype: object


In [ ]:
#stemming
def stemmed_wrapper(term):
  return stemmer.stem(term)
term_dict = {}
for document in soal["Stopwords"]:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------")

for term in term_dict:
  term_dict[term] = stemmed_wrapper(term)
  print(term,":",term_dict[term])

print(term_dict)
print("------")

#apply stemmed term to dataframe
def get_stemmed_term(document):
  return [term_dict[term] for term in document]
soal["Stemmed"] = soal["Stopwords"].apply(get_stemmed_term)
print(soal["Stemmed"])

512
------
aktor : aktor
bertanggung : tanggung
analisis : analisis
prediktif : prediktif
statistik : statistik
tools : tools
analitik : analitik
canggih : canggih
algoritmanya : algoritmanya
fokusnya : fokus
enterpriseexecutive : enterpriseexecutive
is : is
sejarah : sejarah
business : business
inteligent : inteligent
sistem : sistem
pemrosesan : pemrosesan
transaksi : transaksi
konstan : konstan
terlibat : libat
menangani : tangan
pembaruan : baru
basis : basis
data : data
operasional : operasional
komponen : komponen
dikembangkan : kembang
gartner : gartner
menguraikan : urai
perencanaan : rencana
pelaksanaan : laksana
bi : bi
kecuali : kecuali
fokus : fokus
utama : utama
pengembangan : kembang
informasi : informasi
mendukung : dukung
pengambilan : ambil
keputusan : putus
penyediaan : sedia
laporan : lapor
terstruktur : struktur
berkala : kala
intelligent : intelligent
csf : csf
intelligence : intelligence
mengharuskan : harus
cepat : cepat
menerima : terima
perubahan : ubah
beradap

In [ ]:
#save to excel
soal.to_excel(r"/content/drive/MyDrive/PER-TA-AN DUNIAWI/Data/42-01/Preprocessing.xlsx",index=False,header=True)